In [8]:
import os

import pandas as pd

from scripts.generate_outputs import hash_row

In [9]:
output_dir = "../output"
datasets = ['bigbio/meqsum', 'cnn_dailymail', 'full_chat', 'mbpp', 'red_team_chat', 'xsum']
dfs = {}
for dataset_ in datasets:
    dataset = dataset_
    # define output folder and cache csv path
    dataset_dir = dataset.replace("/", "_")
    save_folder = os.path.join(output_dir, dataset_dir)
    os.makedirs(save_folder, exist_ok=True)
    files = os.listdir(save_folder)
    files = [f for f in files if f.endswith(".csv")]
    files = [f for f in files if not f.endswith("embeddings.csv")]
    for f in files:
        model_id = f.split("_predictions.csv")[0]
        model_path = os.path.join(save_folder, f)
        df = pd.read_csv(model_path)
        dfs[model_path] = df

/tmp/ipykernel_2078443/1505285216.py:16: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(model_path)
/tmp/ipykernel_2078443/1505285216.py:16: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(model_path)
/tmp/ipykernel_2078443/1505285216.py:16: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(model_path)
/tmp/ipykernel_2078443/1505285216.py:16: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(model_path)
/tmp/ipykernel_2078443/1505285216.py:16: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(model_path)


In [10]:
len(dfs.keys())

9

In [11]:
dfs.keys()

dict_keys(['../output/bigbio_meqsum/tiiuae-falcon-7b-instruct_predictions.csv', '../output/bigbio_meqsum/tiiuae-falcon-40b-instruct_predictions.csv', '../output/cnn_dailymail/meta-llama-Llama-2-7b-chat-hf_predictions.csv', '../output/full_chat/google-flan-t5-xl_predictions.csv', '../output/full_chat/google-flan-t5-xxl_predictions.csv', '../output/mbpp/codellama-CodeLlama-7b-Instruct-hf_predictions.csv', '../output/red_team_chat/google-flan-t5-xl_predictions.csv', '../output/red_team_chat/google-flan-t5-xxl_predictions.csv', '../output/xsum/meta-llama-Llama-2-7b-chat-hf_predictions.csv'])

## Select top 20 hypotheses

In [31]:
# top 20 hypotheses for xsum/cnn_dailymail and full_chat/red_team_chat
# by summing their scores 
datasets = ['cnn_dailymail',  'xsum', 'full_chat', 'red_team_chat']
cnn_dailymail_hyps = dfs['../output/cnn_dailymail/meta-llama-Llama-2-7b-chat-hf_predictions.csv'].groupby(['hypothesis'])['rougeL'].mean()
xsum_hyps = dfs['../output/xsum/meta-llama-Llama-2-7b-chat-hf_predictions.csv'].groupby(['hypothesis'])['rougeL'].mean()


In [37]:
summed_hyp_scores = xsum_hyps + cnn_dailymail_hyps
# sort by score and select top 20
top_news_hyps = summed_hyp_scores.sort_values(ascending=False).head(20)
print(top_news_hyps)
top_news_hyps = [eval(x)['instruction'] for x in top_news_hyps.index.tolist()]

hypothesis
{'instruction': "Your goal is to provide a summarized version of the document's content in fewer words."}                    0.320253
{'instruction': 'Your goal is to provide shorter versions of the documents that encompass the main ideas.'}                  0.295859
{'instruction': 'Generate concise summaries that provide a quick overview of the document content.'}                         0.288168
{'instruction': 'Create concise summaries that communicate the primary concepts in the documents.'}                          0.287793
{'instruction': 'You are tasked with generating succinct summaries for the given documents.'}                                0.286266
{'instruction': "Your task involves producing compact summaries that convey the documents' core concepts."}                  0.285061
{'instruction': 'Your objective is to create concise summaries of the provided documents.'}                                  0.284100
{'instruction': 'Summarize the documents in a way t

In [38]:
top_news_hyps

["Your goal is to provide a summarized version of the document's content in fewer words.",
 'Your goal is to provide shorter versions of the documents that encompass the main ideas.',
 'Generate concise summaries that provide a quick overview of the document content.',
 'Create concise summaries that communicate the primary concepts in the documents.',
 'You are tasked with generating succinct summaries for the given documents.',
 "Your task involves producing compact summaries that convey the documents' core concepts.",
 'Your objective is to create concise summaries of the provided documents.',
 'Summarize the documents in a way that captures the essence of the original content.',
 'Create brief summaries that capture the essential information contained in the documents.',
 'The aim is to produce abridged summaries of the provided text documents.',
 'Summarize the content of the documents to capture the essential meaning.',
 'Produce succinct and informative summaries for each of the

In [40]:
# repeat for chat datasets
full_chat_hyps = dfs['../output/full_chat/google-flan-t5-xxl_predictions.csv'].groupby(['hypothesis'])['weqweasdas/hh_rlhf_rm_open_llama_3b_eval_reward_med_sigmoid'].mean()
red_team_chat_hyps = dfs['../output/red_team_chat/google-flan-t5-xxl_predictions.csv'].groupby(['hypothesis'])['weqweasdas/hh_rlhf_rm_open_llama_3b_eval_reward_med_sigmoid'].mean()

summed_hyp_scores = full_chat_hyps + red_team_chat_hyps
# sort by score and select top 20
top_chat_hyps = summed_hyp_scores.sort_values(ascending=False).head(20)
print(top_chat_hyps)
top_chat_hyps = [eval(x)['instruction'] for x in top_chat_hyps.index.tolist()]

hypothesis
{'instruction': 'You are a helpful chatbot. Your mission is to provide accurate information, share wisdom, and be friendly.'}                       1.111292
{'instruction': 'You are a helpful chatbot. Your purpose is to provide accurate information, share positivity, and be supportive.'}                 1.083162
{'instruction': "You are a helpful chatbot. You're committed to spreading kindness and providing accurate information."}                            1.073551
{'instruction': 'You are a helpful chatbot. Your purpose is to share insights, resources, and encouragement.'}                                      1.069509
{'instruction': 'You are a helpful chatbot. Your goal is to provide users with accurate information and a friendly experience.'}                    1.064923
{'instruction': 'You are a helpful chatbot. Your focus is on spreading positivity, sharing knowledge, and being kind.'}                             1.062406
{'instruction': 'You are a helpful chatbot. You

In [41]:
top_chat_hyps

['You are a helpful chatbot. Your mission is to provide accurate information, share wisdom, and be friendly.',
 'You are a helpful chatbot. Your purpose is to provide accurate information, share positivity, and be supportive.',
 "You are a helpful chatbot. You're committed to spreading kindness and providing accurate information.",
 'You are a helpful chatbot. Your purpose is to share insights, resources, and encouragement.',
 'You are a helpful chatbot. Your goal is to provide users with accurate information and a friendly experience.',
 'You are a helpful chatbot. Your focus is on spreading positivity, sharing knowledge, and being kind.',
 'You are a helpful chatbot. Your goal is to offer guidance, practical tips, and emotional support.',
 'You are a helpful chatbot. Your goal is to provide users with accurate information, share insights, and encourage well-being.',
 'You are a helpful chatbot. Your purpose is to create connections, offer insights, and encourage positivity.',
 "You a

## Sanity check number generations

In [13]:
# cols = ['text', 'hypothesis', 'dataset', 'model_name_or_path', 'seed', 'task_id']
# df = df.drop_duplicates(subset=cols)
# # assert that all hypotheses have the same number of generations
# assert len(set(df.groupby('hypothesis').size())) == 1

AssertionError: 

In [18]:
# tally up unique hypotheses per dataset
for key in dfs.keys():
    df = dfs[key]
    print(key, df['hypothesis'].nunique())

../output/bigbio_meqsum/tiiuae-falcon-7b-instruct_predictions.csv 50
../output/bigbio_meqsum/tiiuae-falcon-40b-instruct_predictions.csv 50
../output/cnn_dailymail/meta-llama-Llama-2-7b-chat-hf_predictions.csv 40
../output/full_chat/google-flan-t5-xl_predictions.csv 2
../output/full_chat/google-flan-t5-xxl_predictions.csv 50
../output/mbpp/codellama-CodeLlama-7b-Instruct-hf_predictions.csv 30
../output/red_team_chat/google-flan-t5-xl_predictions.csv 2
../output/red_team_chat/google-flan-t5-xxl_predictions.csv 50
../output/xsum/meta-llama-Llama-2-7b-chat-hf_predictions.csv 30


In [19]:
for key in dfs.keys():
    df = dfs[key]
    print(key)
    # number of generations per hypothesis
    print(df['hypothesis'].value_counts())

../output/bigbio_meqsum/tiiuae-falcon-7b-instruct_predictions.csv
hypothesis
{'instruction': "Your goal is to generate a succinct version of the user's question that captures the main points."}                                                              990
{'instruction': "You are tasked with creating a shortened version of the user's question that retains the main ideas.", 'k_shot_idxs': [2, 3]}                                    990
{'instruction': "Your goal is to generate a succinct version of the user's question that captures the main points.", 'k_shot_idxs': [6]}                                          990
{'instruction': "You are tasked with creating a shortened version of the user's question that retains the main ideas.", 'k_shot_idxs': [7, 5]}                                    990
{'instruction': "You are required to produce a concise version of the user's question that preserves the key information.", 'k_shot_idxs': [5]}                                   990
{'instruction

In [22]:
n_total = 8_000
num_hypotheses = 20
# how many hypotheses have 8k generations?
for key in dfs.keys():
    df = dfs[key]
    print(key)
    # number of generations per hypothesis
    value_counts = df['hypothesis'].value_counts()
    print(f'{value_counts[value_counts == n_total].shape[0]}/{num_hypotheses}')

../output/bigbio_meqsum/tiiuae-falcon-7b-instruct_predictions.csv
0/50
../output/bigbio_meqsum/tiiuae-falcon-40b-instruct_predictions.csv
0/50
../output/cnn_dailymail/meta-llama-Llama-2-7b-chat-hf_predictions.csv
16/40
../output/full_chat/google-flan-t5-xl_predictions.csv
0/2
../output/full_chat/google-flan-t5-xxl_predictions.csv
19/50
../output/mbpp/codellama-CodeLlama-7b-Instruct-hf_predictions.csv
0/30
../output/red_team_chat/google-flan-t5-xl_predictions.csv
0/2
../output/red_team_chat/google-flan-t5-xxl_predictions.csv
20/50
../output/xsum/meta-llama-Llama-2-7b-chat-hf_predictions.csv
10/30


In [6]:
# check how many rows are duplicated
cols = ['text', 'hypothesis', 'dataset', 'model_name_or_path', 'seed', 'task_id']
for key in dfs.keys():
    df = dfs[key]
    print(key)
    print(df.shape)
    df['hash'] = df.apply(lambda x: hash_row(x, cols), axis=1)
    print(df.drop_duplicates(subset='hash').shape)
    print()
    dfs[key] = df

../output/bigbio_meqsum/tiiuae-falcon-7b-instruct_predictions.csv
(49500, 21)
(49500, 22)

../output/bigbio_meqsum/tiiuae-falcon-40b-instruct_predictions.csv
(49500, 22)
(49500, 22)

../output/cnn_dailymail/meta-llama-Llama-2-7b-chat-hf_predictions.csv
(176000, 24)
(176000, 24)

../output/full_chat/google-flan-t5-xl_predictions.csv
(40, 21)
(40, 22)

../output/full_chat/google-flan-t5-xxl_predictions.csv
(252000, 22)
(252000, 22)

../output/mbpp/codellama-CodeLlama-7b-Instruct-hf_predictions.csv
(289200, 26)
(289200, 26)

../output/red_team_chat/google-flan-t5-xl_predictions.csv
(40, 21)
(40, 22)

../output/red_team_chat/google-flan-t5-xxl_predictions.csv
(220000, 22)
(220000, 22)

../output/xsum/meta-llama-Llama-2-7b-chat-hf_predictions.csv
(120000, 24)


KeyboardInterrupt: 

In [10]:
# drop duplicate hash values
for key in dfs.keys():
    df = dfs[key]
    print(key)
    print(df.shape)
    df = df.drop_duplicates(subset='hash')
    print(df.shape)
    print()
    dfs[key] = df

../output/bigbio_meqsum/tiiuae-falcon-7b-instruct_predictions.csv
(49500, 22)
(49500, 22)

../output/bigbio_meqsum/tiiuae-falcon-40b-instruct_predictions.csv
(29700, 22)
(29700, 22)

../output/cnn_dailymail/meta-llama-Llama-2-7b-chat-hf_predictions.csv
(60000, 20)
(60000, 20)

../output/full_chat/google-flan-t5-xl_predictions.csv
(40, 22)
(40, 22)

../output/full_chat/google-flan-t5-xxl_predictions.csv
(100000, 22)
(100000, 22)

../output/mbpp/codellama-CodeLlama-7b-Instruct-hf_predictions.csv
(289200, 26)
(289200, 26)

../output/red_team_chat/google-flan-t5-xl_predictions.csv
(40, 22)
(40, 22)

../output/red_team_chat/google-flan-t5-xxl_predictions.csv
(100000, 22)
(100000, 22)

../output/xsum/meta-llama-Llama-2-7b-chat-hf_predictions.csv
(60000, 20)
(60000, 20)



In [11]:
# check that all hypotheses have the same number of examples
for key in dfs.keys():
    df = dfs[key]
    print(key)
    cols = ['text', 'hypothesis', 'dataset', 'model_name_or_path', 'seed', 'task_id']
    # assert that all hypotheses have the same number of generations
    assert len(set(df.groupby(cols).size())) == 1

../output/bigbio_meqsum/tiiuae-falcon-7b-instruct_predictions.csv
../output/bigbio_meqsum/tiiuae-falcon-40b-instruct_predictions.csv
../output/cnn_dailymail/meta-llama-Llama-2-7b-chat-hf_predictions.csv
../output/full_chat/google-flan-t5-xl_predictions.csv
../output/full_chat/google-flan-t5-xxl_predictions.csv
../output/mbpp/codellama-CodeLlama-7b-Instruct-hf_predictions.csv
../output/red_team_chat/google-flan-t5-xl_predictions.csv
../output/red_team_chat/google-flan-t5-xxl_predictions.csv
../output/xsum/meta-llama-Llama-2-7b-chat-hf_predictions.csv


In [4]:
# summarize what we have
for key in dfs.keys():
    df = dfs[key]
    print(key)
    # number of unique hypotheses
    print(f'Number of hypotheses: {df["hypothesis"].nunique()}. Generations per hypothesis: {df["hypothesis"].value_counts()[0]:,}')

../output/bigbio_meqsum/tiiuae-falcon-7b-instruct_predictions.csv
Number of hypotheses: 50. Generations per hypothesis: 990
../output/bigbio_meqsum/tiiuae-falcon-40b-instruct_predictions.csv
Number of hypotheses: 30. Generations per hypothesis: 990
../output/cnn_dailymail/meta-llama-Llama-2-7b-chat-hf_predictions.csv
Number of hypotheses: 40. Generations per hypothesis: 8,000
../output/full_chat/google-flan-t5-xl_predictions.csv
Number of hypotheses: 2. Generations per hypothesis: 20
../output/full_chat/google-flan-t5-xxl_predictions.csv
Number of hypotheses: 50. Generations per hypothesis: 8,000
../output/mbpp/codellama-CodeLlama-7b-Instruct-hf_predictions.csv
Number of hypotheses: 30. Generations per hypothesis: 9,640
../output/red_team_chat/google-flan-t5-xl_predictions.csv
Number of hypotheses: 2. Generations per hypothesis: 20
../output/red_team_chat/google-flan-t5-xxl_predictions.csv
Number of hypotheses: 50. Generations per hypothesis: 8,000
../output/xsum/meta-llama-Llama-2-7b-

/tmp/ipykernel_1810323/3566897220.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'Number of hypotheses: {df["hypothesis"].nunique()}. Generations per hypothesis: {df["hypothesis"].value_counts()[0]:,}')
/tmp/ipykernel_1810323/3566897220.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'Number of hypotheses: {df["hypothesis"].nunique()}. Generations per hypothesis: {df["hypothesis"].value_counts()[0]:,}')
/tmp/ipykernel_1810323/3566897220.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent

In [8]:
# # drop hash column and save back to disk
# for key in dfs.keys():
#     df = dfs[key]
#     print(key)
#     print(df.shape)
#     df = df.drop(columns=['hash'])
#     print(df.shape)
#     df.to_csv(key, index=False)
#     print()

In [13]:
# estimate time to run all examples for single prompt:

In [3]:
from datasets import load_dataset

# Load all helpfulness/harmless subsets (share the same schema)
dataset = load_dataset("Anthropic/hh-rlhf")
print('full_chat')
print(dataset)
print(f'Approx. time to run all examples for full_chat single prompt for {len(dataset["test"]):,} examples: {len(dataset["test"]) * 0.12 / 60:.2f} minutes.')

# Load the red teaming subset
dataset = load_dataset("Anthropic/hh-rlhf", data_dir="red-team-attempts")
print('red_team_chat')
print(dataset)
print(f'Approx. time to run all examples for red_team_chat single prompt for {len(dataset["train"]):,} examples: {len(dataset["train"]) * 0.12 / 60:.2f} minutes.')


full_chat
DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected'],
        num_rows: 160800
    })
    test: Dataset({
        features: ['chosen', 'rejected'],
        num_rows: 8552
    })
})
Approx. time to run all examples for full_chat single prompt for 8,552 examples: 17.10 minutes.
red_team_chat
DatasetDict({
    train: Dataset({
        features: ['is_upworker', 'model_type', 'num_params', 'transcript', 'rating', 'red_team_member_id', 'task_descripton_harmlessness_score', 'tags', 'min_harmlessness_score_transcript', 'task_description'],
        num_rows: 38961
    })
})
Approx. time to run all examples for red_team_chat single prompt for 38,961 examples: 77.92 minutes.
